In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/GOOG.csv')
data.head()

In [ ]:
data = data['Close'].values

# Reshape the data to a 2D array for scaling
data = data.reshape(-1, 1)

# Normalize the stock prices to range between 0 and 1
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [ ]:
# Define the number of days to use for prediction (look-back window)
look_back = 60

# Prepare the data
X = []
y = []

for i in range(look_back, len(scaled_data)):
  X.append(scaled_data[i - look_back:i, 0]) # Use the last 60 days of data
  y.append(scaled_data[i, 0]) # The price on the current day (t)

# Convert the data into numpy arrays
X = np.array(X)
y = np.array(y)

# Reshape X to be 3D: [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

In [ ]:
# Build the RNN model using LSTM
model = Sequential()

# Add an LSTM layer
model.add(LSTM(units=50, return_sequences=False, input_shape=(X.shape[1], 1)))

# Add a dense output layer
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(X, y, epochs=5, batch_size=32)

In [ ]:
# Predict stock prices on the test data (the last 60 days)
predicted_stock_price = model.predict(X)

# Inverse transform to get the actual stock prices
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)

In [ ]:
# Plot the results
plt.plot(data, color='blue', label='Actual Google Stock Price')
plt.plot(np.arange(look_back, len(data)), predicted_stock_price, color='red', label='Predicted GoogleStock Price')
plt.title('Google Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

In [ ]:
#Evaluate the model
from sklearn.metrics import mean_squared_error

# Calculate the Mean Squared Error
mse = mean_squared_error(y, predicted_stock_price)
print(f"Mean Squared Error: {mse}")